In [1]:
# Import libraries
import os

import pandas as pd
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.nn import Softmax

In [2]:
df_news = pd.read_csv(os.path.join('..', 'data', 'RedditNews_preprocessed.csv'))

In [3]:
# Check if a GPU is available and if not, use a CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained model and tokenizer (FinBERT from ProsusAI)
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

# Move the model to the GPU if one is available
model = model.to(device)

# Put model in evaluation mode
model.eval()

def get_sentiment_FinBert(texts, batch_size=32):
    sentiments = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs = inputs.to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        probs = Softmax(dim=1)(outputs.logits)
        probs = probs.cpu()
        sentiments.extend(probs.tolist())
    return sentiments

# Apply the function to the headlines
sentiments = get_sentiment_FinBert(df_news['News'].tolist())
df_news[['sentiment_FinBert_positive', 'sentiment_FinBert_negative', 'sentiment_FinBert_neutral']] = pd.DataFrame(sentiments)

KeyboardInterrupt: 

In [ ]:
df_news.to_csv(os.path.join('..', 'data', 'RedditNews_FinBert.csv'), index=False)

In [ ]:
df_= pd.read_csv(os.path.join('..', 'data', 'RedditNews_FinBert.csv'))

In [5]:
df_

,Unnamed: 0,Date,News,News_Preprocessed,sentiment_FinBert_positive,sentiment_FinBert_negative,sentiment_FinBert_neutral
0,0,2016-07-01,A 117-year-old woman in Mexico City finally re...,117yearold woman mexico city finally received ...,0.040008,0.112128,0.847864
1,1,2016-07-01,IMF chief backs Athens as permanent Olympic host,imf chief back athens permanent olympic host,0.180750,0.014429,0.804821
2,2,2016-07-01,"The president of France says if Brexit won, so...",president france say brexit donald trump,0.189452,0.013524,0.797024
3,3,2016-07-01,British Man Who Must Give Police 24 Hours' Not...,british man must give police 24 hour notice se...,0.020568,0.754183,0.225249
4,4,2016-07-01,100+ Nobel laureates urge Greenpeace to stop o...,100 nobel laureate urge greenpeace stop opposi...,0.116268,0.077855,0.805877
...,...,...,...,...,...,...,...
73603,73603,2008-06-08,b'Man goes berzerk in Akihabara and stabs ever...,man go berzerk akihabara stab everyone nearby ...,0.024737,0.718907,0.256357
73604,73604,2008-06-08,b'Threat of world AIDS pandemic among heterose...,threat world aid pandemic among heterosexual r...,0.123800,0.056227,0.819974
73605,73605,2008-06-08,b'Angst in Ankara: Turkey Steers into a Danger...,angst ankara turkey steer dangerous identity c...,0.032466,0.917706,0.049828
73606,73606,2008-06-08,"b""UK: Identity cards 'could be used to spy on ...",buk identity card could used spy people new ch...,0.052249,0.106890,0.840861
